In [1]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"  # or "tensorflow" or "torch"

import keras_nlp
import tensorflow as tf
import keras


KeyboardInterrupt



In [ ]:
PRETRAINING_BATCH_SIZE = 32 
FINETUNING_BATCH_SIZE = 8
SEQ_LENGTH = 512 
MASK_RATE = 0.15 
PREDICTIONS_PER_SEQ = int(SEQ_LENGTH*MASK_RATE)+1 


NUM_LAYERS = 3 
MODEL_DIM = 512 
INTERMEDIATE_DIM = MODEL_DIM*2 
NUM_HEADS = 4 
DROPOUT = 0.1
NORM_EPSILON = 1e-5 


PRETRAINING_LEARNING_RATE = 0.1e-3 
PRETRAINING_EPOCHS = 50 
FINETUNING_LEARNING_RATE = 0.01e-3
FINETUNING_EPOCHS = 500

In [ ]:
import pandas as pd
train_Data = pd.read_json('CNewSum_v2.jsonl', lines=True ) 
print(train_Data.head())

In [ ]:
validation_Data = pd.read_json('prose.jsonl', lines=True ) 

In [ ]:
train_text =[]
for index, row in train_Data.iterrows():
    train_text.append(row["text"])


In [ ]:
validation_text =[]
for index, row in validation_Data.iterrows():
    validation_text.append(row["text"])


In [ ]:
train_ds = (
     tf.data.Dataset.from_tensor_slices(train_text) 
    .filter(lambda x: tf.strings.length(x) > 100) 
    .batch(PRETRAINING_BATCH_SIZE)
)


In [ ]:
validation_ds = (
     tf.data.Dataset.from_tensor_slices(validation_text) 
    .filter(lambda x: tf.strings.length(x) > 100) 
    .batch(PRETRAINING_BATCH_SIZE)
)

In [ ]:
tokenizer = keras_nlp.tokenizers.WordPieceTokenizer(
    vocabulary="bert_base_chinese/vocab.txt", 
    sequence_length=SEQ_LENGTH, 
    lowercase=True, 
    strip_accents=True,
)


masker = keras_nlp.layers.MaskedLMMaskGenerator( 

    vocabulary_size=tokenizer.vocabulary_size(),
    mask_selection_rate=MASK_RATE, 
    mask_selection_length=PREDICTIONS_PER_SEQ, 
    mask_token_id=tokenizer.token_to_id("[MASK]"), 
    
    unselectable_token_ids=[0]
) 

def preprocess(inputs): 
    inputs = tokenizer(inputs)

    outputs = masker(inputs)

    features = {
        "token_ids": outputs["token_ids"], 
        "mask_positions": outputs["mask_positions"], 
    }
    labels = outputs["mask_ids"] 
    weights = outputs["mask_weights"] 
    return features, labels, weights 


pretrain_ds = train_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE #num_parallel_calls 参数的作用是在多个CPU 内核中并行调用map()
).prefetch(tf.data.AUTOTUNE) #如果你不确定使用多大的缓冲区，可以尝试dataset.prefetch(tf.data.AUTOTUNE) 选项，它会为你选择缓冲区大小。
prevalidation_ds = validation_ds.map(
    preprocess, num_parallel_calls=tf.data.AUTOTUNE 
).prefetch(tf.data.AUTOTUNE) 






In [ ]:
#定义模型
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype="int32") 

# Embed our tokens with a positional embedding.
embedding_layer = keras_nlp.layers.TokenAndPositionEmbedding(
    vocabulary_size=tokenizer.vocabulary_size(),
    sequence_length=SEQ_LENGTH, 
    embedding_dim=MODEL_DIM, 
)
outputs = embedding_layer(inputs)

# Apply layer normalization and dropout to the embedding.
outputs = keras.layers.LayerNormalization(epsilon=NORM_EPSILON)(outputs) #https://keras.io/api/layers/normalization_layers/layer_normalization/
outputs = keras.layers.Dropout(rate=DROPOUT)(outputs)

# Add a number of encoder blocks
for i in range(NUM_LAYERS): # NUM_LAYERS = 3 
    outputs = keras_nlp.layers.TransformerEncoder( #https://keras.io/api/keras_nlp/modeling_layers/transformer_encoder/
        intermediate_dim=INTERMEDIATE_DIM, #the hidden size of feedforward network
        num_heads=NUM_HEADS, 
        dropout=DROPOUT, #DROPOUT = 0.1
        layer_norm_epsilon=NORM_EPSILON, #NORM_EPSILON = 1e-5
    )(outputs)

encoder_model = keras.Model(inputs, outputs)
encoder_model.summary()



In [ ]:
# Create the pretraining model by attaching a masked language model head.
inputs = {
    "token_ids": keras.Input(shape=(SEQ_LENGTH,), dtype="int32", name="token_ids"),
    "mask_positions": keras.Input(
        shape=(PREDICTIONS_PER_SEQ,), dtype="int32", name="mask_positions"
    ),
}

# Encode the tokens.
encoded_tokens = encoder_model(inputs["token_ids"])

outputs = keras_nlp.layers.MaskedLMHead( #https://keras.io/api/keras_nlp/modeling_layers/masked_lm_head/
    token_embedding=embedding_layer.token_embedding,
    activation="softmax", 
)(encoded_tokens, mask_positions=inputs["mask_positions"])





# Define and compile our pretraining model.
pretraining_model = keras.Model(inputs, outputs)
pretraining_model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.AdamW(PRETRAINING_LEARNING_RATE),
    weighted_metrics=["sparse_categorical_accuracy"],
    jit_compile=True, 
)


# Pretrain the model on our wiki text dataset.
history=pretraining_model.fit( 
    pretrain_ds, 
    validation_data=prevalidation_ds,
    #validation_split=0.2, 不支持
    epochs=PRETRAINING_EPOCHS, 
)

# Save this base model for further finetuning.
encoder_model.save("encoder_model.keras") #生成文件“encoder_model.keras”

with open(os.path.join('history01.txt'), 'w') as file_object:
    file_object.write(str(history.history))


In [2]:
with open(os.path.join('history01.txt')) as file_object:
    history=file_object.read()

history={'loss': [4.215114116668701, 2.701319456100464, 2.053736686706543, 1.8690412044525146, 1.764220118522644, 1.6849459409713745, 1.6250251531600952, 1.57936429977417, 1.543154001235962, 1.5141745805740356, 1.4890203475952148, 1.466396689414978, 1.4466533660888672, 1.4295892715454102, 1.4145846366882324, 1.3997644186019897, 1.3851103782653809, 1.373183250427246, 1.3614057302474976, 1.3495548963546753, 1.339219093322754, 1.3301010131835938, 1.3215973377227783, 1.3135480880737305, 1.3056963682174683, 1.2995177507400513, 1.2929863929748535, 1.2854880094528198, 1.2803492546081543, 1.2751312255859375, 1.2690489292144775, 1.2646287679672241, 1.259752869606018, 1.2543983459472656, 1.2503721714019775, 1.2460603713989258, 1.2424672842025757, 1.238459587097168, 1.2334705591201782, 1.2299669981002808, 1.227776288986206, 1.2239623069763184, 1.2206940650939941, 1.2178676128387451, 1.2141826152801514, 1.2116522789001465, 1.2078624963760376, 1.2055448293685913, 1.2032933235168457, 1.2007561922073364], 'sparse_categorical_accuracy': [0.16027024388313293, 0.38344040513038635, 0.4990787208080292, 0.5313647389411926, 0.5504758954048157, 0.565305769443512, 0.5769724249839783, 0.5858662724494934, 0.5930466651916504, 0.5986890196800232, 0.6038194298744202, 0.6083085536956787, 0.6122370958328247, 0.6159961223602295, 0.6189829707145691, 0.6221186518669128, 0.6253342628479004, 0.6279077529907227, 0.630246639251709, 0.6329491138458252, 0.6351562142372131, 0.6370694041252136, 0.6389247179031372, 0.6407492756843567, 0.6424031257629395, 0.6437627673149109, 0.6451208591461182, 0.6468157172203064, 0.6480141282081604, 0.6490411758422852, 0.650374710559845, 0.6512444019317627, 0.6523885726928711, 0.6535740494728088, 0.6545524001121521, 0.6555305123329163, 0.6562032699584961, 0.6571522355079651, 0.6581202745437622, 0.6590268015861511, 0.6595544219017029, 0.6603213548660278, 0.6611647605895996, 0.6616901159286499, 0.6626507043838501, 0.6630846858024597, 0.6640158295631409, 0.6644671559333801, 0.6648055911064148, 0.6656462550163269], 'val_loss': [4.90177583694458, 3.851116418838501, 3.5851528644561768, 3.4407565593719482, 3.3502163887023926, 3.2681453227996826, 3.1932287216186523, 3.1795926094055176, 3.1003165245056152, 3.1213464736938477, 3.058871030807495, 3.0433359146118164, 3.0275235176086426, 2.995920181274414, 2.972259283065796, 2.9679453372955322, 2.9501023292541504, 2.9427359104156494, 2.929672956466675, 2.888206720352173, 2.905555486679077, 2.886664867401123, 2.855043649673462, 2.8553578853607178, 2.8280370235443115, 2.8528902530670166, 2.8349502086639404, 2.8457298278808594, 2.8040852546691895, 2.8390424251556396, 2.8028173446655273, 2.7956788539886475, 2.806994676589966, 2.7979564666748047, 2.7669739723205566, 2.758753538131714, 2.7611114978790283, 2.7830681800842285, 2.7643401622772217, 2.7428317070007324, 2.7211341857910156, 2.7603254318237305, 2.7315785884857178, 2.7580201625823975, 2.7397449016571045, 2.7435076236724854, 2.7199583053588867, 2.7224066257476807, 2.706904649734497, 2.7114832401275635], 'val_sparse_categorical_accuracy': [0.19230128824710846, 0.33604946732521057, 0.3609560430049896, 0.3725062906742096, 0.3834265172481537, 0.391301691532135, 0.40091991424560547, 0.4022219181060791, 0.4118821918964386, 0.41104209423065186, 0.4178042709827423, 0.4181613028049469, 0.420576274394989, 0.4254274070262909, 0.4276534914970398, 0.4302995800971985, 0.42966949939727783, 0.43069854378700256, 0.4348776340484619, 0.43748173117637634, 0.4371036887168884, 0.438951700925827, 0.44428589940071106, 0.44359290599823, 0.44825494289398193, 0.4449578523635864, 0.44756191968917847, 0.44863295555114746, 0.44964098930358887, 0.4450208842754364, 0.4506489634513855, 0.452392041683197, 0.45090100169181824, 0.45060697197914124, 0.455500066280365, 0.45835620164871216, 0.45783117413520813, 0.4543241262435913, 0.4575371742248535, 0.4573691785335541, 0.46255621314048767, 0.4561511278152466, 0.46056121587753296, 0.459175169467926, 0.45904919505119324, 0.460540235042572, 0.46392130851745605, 0.46247225999832153, 0.46589532494544983, 0.46471938490867615]}

import matplotlib.pyplot as plt
a=28
acc = history['sparse_categorical_accuracy']#[0:a]
val_acc = history['val_sparse_categorical_accuracy']#[0:a]
loss = history['loss']#[0:a]
val_loss = history['val_loss']#[0:a]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: 'history01.txt'

In [ ]:
#以上是预训练，下面是问答头部模型
# Reload the encoder model from disk so we can restart fine-tuning from scratch.
encoder_model = keras.models.load_model("encoder_model.keras", compile=False) 

from tensorflow.keras import layers
inputs = keras.Input(shape=(SEQ_LENGTH,), dtype="int32") 
encoded_tokens = encoder_model(inputs)[0] 
start_logits = layers.Dense(1, name="start_logit", use_bias=False)(encoded_tokens) 
start_logits = layers.Flatten()(start_logits)
end_logits = layers.Dense(1, name="end_logit", use_bias=False)(encoded_tokens)
end_logits = layers.Flatten()(end_logits)
start_probs = layers.Activation(keras.activations.softmax)(start_logits) 
end_probs = layers.Activation(keras.activations.softmax)(end_logits)
finetuning_model = keras.Model( inputs, outputs=[start_probs, end_probs] )
finetuning_model.summary()

loss = keras.losses.SparseCategoricalCrossentropy(from_logits=False) 
optimizer=keras.optimizers.AdamW(FINETUNING_LEARNING_RATE)
finetuning_model.compile(optimizer=optimizer, loss=[loss, loss], metrics=['accuracy','accuracy']) 

In [ ]:
#文本格式的数据集
df = pd.read_excel('bridge QA 2024.07.25.xlsx')
print(df.head())
print(df.info(),'\n')

import numpy as np
def split_data(data, validate_number=20, test_number=6): #这里数值是样本数而不是比例
    shuffled_indices = np.random.permutation(len(data))
    test_indices = shuffled_indices[:test_number]
    validate_indices = shuffled_indices[test_number:test_number+validate_number]
    train_indices = shuffled_indices[test_number+validate_number:]
    return data.iloc[train_indices], data.iloc[validate_indices], data.iloc[test_indices]
train_set, validate_set, test_set = split_data(df, validate_number=20, test_number=6) #这里数值是样本数而不是比例


In [ ]:
from tokenizers import BertWordPieceTokenizer 
tokenizer = BertWordPieceTokenizer("bert_base_chinese/vocab.txt", lowercase=True)
max_len=512 #样本最大长度

In [ ]:
def start_and_end_token_idx(context_ids, answer_ids, index_of_answer):
    answer_ids=answer_ids[1:-1]
    answer_ids_len=len(answer_ids)
    answer_ids = ' '.join([str(i) for i in answer_ids]) 
    context_ids = ' '.join([str(i) for i in context_ids]) 
    context_ids=context_ids.replace(answer_ids,"#") 
    context_ids=context_ids.split() 
    
    times=0
    for i,value in enumerate(context_ids):
        if value=="#":
            if times==index_of_answer:
                return i+(answer_ids_len-1)*times, i+(answer_ids_len-1)*times+(answer_ids_len-1)   
            times +=1


In [ ]:
class BridgeExample: 
     def __init__(self, question_text, context_text, answer_text, index_of_answer):
         self.question = question_text
         self.context = context_text
         self.answer = answer_text
         self.index_of_answer = index_of_answer 
         self.skip = False 
     def preprocess(self):
         
         context = " ".join(str(self.context).split()) 
         question = " ".join(str(self.question).split()) 
         answer = " ".join(str(self.answer).split())

         tokenized_question = tokenizer.encode(question)
         tokenized_context = tokenizer.encode(context)
         tokenized_answer = tokenizer.encode(answer)
 
         start_token_idx,end_token_idx=start_and_end_token_idx(tokenized_context.ids, tokenized_answer.ids, self.index_of_answer)

         input_ids = tokenized_context.ids + tokenized_question.ids[1:] 
         token_type_ids = [0] * len(tokenized_context.ids) + [1] * len(tokenized_question.ids[1:]) #样本
         attention_mask = [1] * len(input_ids) #样本

         padding_length = max_len - len(input_ids)
         if padding_length > 0: #用0填充到设定的最大长度
             input_ids = input_ids + ([0] * padding_length) 
             attention_mask = attention_mask + ([0] * padding_length) 
             token_type_ids = token_type_ids + ([0] * padding_length)
         elif padding_length < 0: #超过设定的最大长度，则做取消这个样本的标记
             self.skip = True
             print("There is an example skip because of padding_length < 0")
             return
         
         #存储样本、标签
         self.input_ids = input_ids #样本
         self.start_token_idx = start_token_idx #标签
         self.end_token_idx = end_token_idx #标签



In [ ]:
def create_bridge_examples(raw_data): 
    bridge_examples = [] #元素是类
    for i in range( len(raw_data["context"]) ):
        context_text = raw_data.iloc[i]["context"] #上下文
        question_text = raw_data.iloc[i]["question"] #问题
        answer_text = raw_data.iloc[i]["answer"] #回答
        index_of_answer = raw_data.iloc[i]["index_of_answer"] #回答的索引（上下文存在多个与答案相同的片段。索引从0开始计数）

        bridge_eg = BridgeExample(question_text, context_text, answer_text, index_of_answer)
        bridge_eg.preprocess() 
        bridge_examples.append(bridge_eg)
    return bridge_examples 

def create_inputs_targets(bridge_examples): 
     dataset_dict = {
         "input_ids": [],#样本的成员
         "start_token_idx": [],#标签的成员
         "end_token_idx": [],#标签的成员
     }
     for item in bridge_examples:
         if item.skip == False: #有效样本
             for key in dataset_dict:
                 dataset_dict[key].append(getattr(item, key)) 

     for key in dataset_dict:
         dataset_dict[key] = np.array(dataset_dict[key])


     x = dataset_dict["input_ids"]#样本
     y = [ dataset_dict["start_token_idx"], dataset_dict["end_token_idx"] ]#标签
     return x, y

In [ ]:

train_bridge_examples = create_bridge_examples(train_set) #训练集
x_train, y_train = create_inputs_targets(train_bridge_examples) 
print(f"{len(train_bridge_examples)} training points created.")

eval_bridge_examples = create_bridge_examples(validate_set) #验证集
x_eval, y_eval = create_inputs_targets(eval_bridge_examples)
print(f"{len(eval_bridge_examples)} evaLuation points created.")

test_bridge_examples = create_bridge_examples(test_set) #测试集
x_test, y_test = create_inputs_targets(test_bridge_examples)
print(f"{len(test_bridge_examples)} test points created.")



In [ ]:

def predict_result(x_data, y_data, show=False):
    pred_start, pred_end = finetuning_model.predict(x_data) 
    count = 0
    for idx, (start, end) in enumerate(zip(pred_start, pred_end)):
        start = np.argmax(start) 
        end = np.argmax(end)
        if start == y_data[0][idx] and end == y_data[1][idx]:
            count += 1 
    acc = count / len(y_data[0]) 
    print('样本数量=',len(y_data[0]))
    print(f"精度={acc:.0%}", "\n") 
    if show:
        for num_of_example in range( len(y_data[0]) ):
            context_AND_question=tokenizer.decode(x_data[num_of_example]) 
            print('context_AND_question=',context_AND_question)
            label=tokenizer.decode(x_data[num_of_example][y_data[0][num_of_example] : y_data[1][num_of_example]+1]) 
            print('label=',y_data[0][num_of_example] , y_data[1][num_of_example],"\n",label)
            answer_start = np.argmax(pred_start[num_of_example])
            answer_end = np.argmax(pred_end[num_of_example])
            label=tokenizer.decode(x_data[num_of_example][answer_start : answer_end+1]) 
            print('预测=',answer_start,answer_end,"\n",label, "\n")

        


In [ ]:

history=finetuning_model.fit( 
    x_train, y_train,
    validation_data=(x_eval, y_eval),
    epochs=FINETUNING_EPOCHS,
    batch_size=FINETUNING_BATCH_SIZE,
)
finetuning_model.save_weights('finetuning_model.weights.h5')

with open(os.path.join('history02.txt'), 'w') as file_object:
    file_object.write(str(history.history))


In [ ]:
with open(os.path.join('history02.txt')) as file_object:
    history=file_object.read()

history={'activation_1_accuracy': [0.0, 0.009999999776482582, 0.04500000178813934, 0.10000000149011612, 0.05999999865889549, 0.0949999988079071, 0.08500000089406967, 0.10999999940395355, 0.125, 0.1550000011920929, 0.125, 0.1550000011920929, 0.17499999701976776, 0.12999999523162842, 0.17499999701976776, 0.18000000715255737, 0.14499999582767487, 0.20499999821186066, 0.20999999344348907, 0.18000000715255737, 0.2199999988079071, 0.2549999952316284, 0.22499999403953552, 0.1899999976158142, 0.23000000417232513, 0.26499998569488525, 0.27000001072883606, 0.2800000011920929, 0.2750000059604645, 0.3400000035762787, 0.36500000953674316, 0.2849999964237213, 0.375, 0.3799999952316284, 0.3799999952316284, 0.41499999165534973, 0.4699999988079071, 0.3700000047683716, 0.4000000059604645, 0.4699999988079071, 0.5, 0.4749999940395355, 0.49000000953674316, 0.5249999761581421, 0.5099999904632568, 0.48500001430511475, 0.5649999976158142, 0.5649999976158142, 0.6000000238418579, 0.6000000238418579, 0.5299999713897705, 0.6499999761581421, 0.6200000047683716, 0.6349999904632568, 0.6549999713897705, 0.6299999952316284, 0.6299999952316284, 0.6499999761581421, 0.6650000214576721, 0.6399999856948853, 0.675000011920929, 0.6499999761581421, 0.6850000023841858, 0.7300000190734863, 0.7099999785423279, 0.6899999976158142, 0.6549999713897705, 0.7099999785423279, 0.75, 0.7250000238418579, 0.7350000143051147, 0.7250000238418579, 0.7549999952316284, 0.7350000143051147, 0.7799999713897705, 0.7599999904632568, 0.7850000262260437, 0.7599999904632568, 0.7799999713897705, 0.7749999761581421, 0.7900000214576721, 0.7799999713897705, 0.7300000190734863, 0.8149999976158142, 0.7749999761581421, 0.800000011920929, 0.8100000023841858, 0.800000011920929, 0.8199999928474426, 0.7850000262260437, 0.8199999928474426, 0.8399999737739563, 0.8550000190734863, 0.800000011920929, 0.8349999785423279, 0.8450000286102295, 0.8050000071525574, 0.7850000262260437, 0.8199999928474426, 0.8700000047683716, 0.8349999785423279, 0.7850000262260437, 0.8299999833106995, 0.8500000238418579, 0.8550000190734863, 0.8600000143051147, 0.8799999952316284, 0.8500000238418579, 0.9049999713897705, 0.8349999785423279, 0.8550000190734863, 0.8450000286102295, 0.8550000190734863, 0.8899999856948853, 0.8500000238418579, 0.8949999809265137, 0.8899999856948853, 0.8600000143051147, 0.9100000262260437, 0.8299999833106995, 0.8849999904632568, 0.875, 0.8799999952316284, 0.8899999856948853, 0.8650000095367432, 0.8600000143051147, 0.8700000047683716, 0.8899999856948853, 0.8999999761581421, 0.8650000095367432, 0.8899999856948853, 0.8999999761581421, 0.9100000262260437, 0.8700000047683716, 0.8899999856948853, 0.8799999952316284, 0.9150000214576721, 0.8849999904632568, 0.925000011920929, 0.8999999761581421, 0.9049999713897705, 0.9300000071525574, 0.8999999761581421, 0.8849999904632568, 0.8949999809265137, 0.8849999904632568, 0.9300000071525574, 0.9200000166893005, 0.8999999761581421, 0.9399999976158142, 0.9200000166893005, 0.8999999761581421, 0.9049999713897705, 0.9200000166893005, 0.8999999761581421, 0.9399999976158142, 0.9150000214576721, 0.9449999928474426, 0.9049999713897705, 0.9300000071525574, 0.8999999761581421, 0.9350000023841858, 0.9449999928474426, 0.9350000023841858, 0.9350000023841858, 0.925000011920929, 0.9350000023841858, 0.9300000071525574, 0.9300000071525574, 0.9200000166893005, 0.9049999713897705, 0.9350000023841858, 0.9399999976158142, 0.9300000071525574, 0.9549999833106995, 0.9549999833106995, 0.8999999761581421, 0.9200000166893005, 0.9350000023841858, 0.9350000023841858, 0.8999999761581421, 0.9449999928474426, 0.9449999928474426, 0.949999988079071, 0.9350000023841858, 0.9649999737739563, 0.9449999928474426, 0.9399999976158142, 0.9350000023841858, 0.9750000238418579, 0.9399999976158142, 0.9350000023841858, 0.9549999833106995, 0.9700000286102295, 0.9549999833106995, 0.9599999785423279, 0.9549999833106995, 0.9599999785423279, 0.925000011920929, 0.9399999976158142, 0.9649999737739563, 0.9449999928474426, 0.9399999976158142, 0.9350000023841858, 0.9449999928474426, 0.9549999833106995, 0.9399999976158142, 0.9649999737739563, 0.949999988079071, 0.9399999976158142, 0.9549999833106995, 0.9800000190734863, 0.9399999976158142, 0.9800000190734863, 0.9549999833106995, 0.9700000286102295, 0.9750000238418579, 0.9599999785423279, 0.9900000095367432, 0.9549999833106995, 0.9649999737739563, 0.9549999833106995, 0.9549999833106995, 0.9900000095367432, 0.9750000238418579, 0.9750000238418579, 0.9700000286102295, 0.9700000286102295, 0.9599999785423279, 0.9599999785423279, 0.9649999737739563, 0.9549999833106995, 0.9549999833106995, 0.9599999785423279, 0.9549999833106995, 0.9649999737739563, 0.9700000286102295, 0.9700000286102295, 0.9649999737739563, 0.9649999737739563, 0.9750000238418579, 0.9549999833106995, 0.9599999785423279, 0.9800000190734863, 0.9750000238418579, 0.9700000286102295, 0.9599999785423279, 0.9700000286102295, 0.9800000190734863, 0.9700000286102295, 0.9750000238418579, 0.9750000238418579, 0.9649999737739563, 0.9700000286102295, 0.9599999785423279, 0.9800000190734863, 0.9750000238418579, 0.9700000286102295, 0.9800000190734863, 0.9900000095367432, 0.9599999785423279, 0.9549999833106995, 0.9800000190734863, 0.9700000286102295, 0.9700000286102295, 0.9800000190734863, 0.9649999737739563, 0.9549999833106995, 0.9750000238418579, 0.9750000238418579, 0.9800000190734863, 0.9800000190734863, 0.9750000238418579, 0.9649999737739563, 0.9900000095367432, 0.9700000286102295, 0.9750000238418579, 0.9599999785423279, 0.9700000286102295, 0.9850000143051147, 0.9850000143051147, 0.9750000238418579, 0.9649999737739563, 0.9850000143051147, 0.9549999833106995, 0.9800000190734863, 0.9750000238418579, 0.9750000238418579, 0.9850000143051147, 0.9750000238418579, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9850000143051147, 1.0, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 0.9850000143051147, 0.9850000143051147, 0.9800000190734863, 0.9900000095367432, 0.9900000095367432, 0.9700000286102295, 0.9800000190734863, 0.9950000047683716, 0.9950000047683716, 0.9700000286102295, 0.9850000143051147, 0.9800000190734863, 0.9850000143051147, 0.9850000143051147, 0.9800000190734863, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 0.9850000143051147, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9900000095367432, 0.9900000095367432, 0.9800000190734863, 0.9950000047683716, 0.9750000238418579, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 1.0, 0.9950000047683716, 0.9800000190734863, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9850000143051147, 0.9800000190734863, 1.0, 0.9900000095367432, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9750000238418579, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 0.9800000190734863, 1.0, 0.9900000095367432, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 1.0, 1.0, 0.9900000095367432, 1.0, 0.9950000047683716, 1.0, 0.9850000143051147, 0.9950000047683716, 1.0, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9900000095367432, 1.0, 0.9900000095367432, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9850000143051147, 0.9850000143051147, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 1.0, 1.0, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9850000143051147, 1.0, 0.9950000047683716, 0.9900000095367432, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9900000095367432, 1.0, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 1.0, 1.0, 1.0, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 1.0, 0.9850000143051147, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9900000095367432, 0.9950000047683716, 1.0, 1.0, 1.0, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 1.0, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716], 'activation_accuracy': [0.009999999776482582, 0.03999999910593033, 0.07000000029802322, 0.05999999865889549, 0.0949999988079071, 0.07000000029802322, 0.10499999672174454, 0.10000000149011612, 0.14000000059604645, 0.12999999523162842, 0.14000000059604645, 0.17000000178813934, 0.11999999731779099, 0.13500000536441803, 0.13500000536441803, 0.1899999976158142, 0.19499999284744263, 0.17000000178813934, 0.18000000715255737, 0.2199999988079071, 0.2199999988079071, 0.23499999940395355, 0.1850000023841858, 0.23499999940395355, 0.20499999821186066, 0.27000001072883606, 0.2800000011920929, 0.24500000476837158, 0.27000001072883606, 0.3149999976158142, 0.3050000071525574, 0.35499998927116394, 0.3799999952316284, 0.38999998569488525, 0.3799999952316284, 0.4699999988079071, 0.42500001192092896, 0.4350000023841858, 0.4350000023841858, 0.5149999856948853, 0.5299999713897705, 0.4950000047683716, 0.5199999809265137, 0.5199999809265137, 0.5649999976158142, 0.4950000047683716, 0.6399999856948853, 0.6000000238418579, 0.5899999737739563, 0.5899999737739563, 0.5849999785423279, 0.6399999856948853, 0.6349999904632568, 0.6399999856948853, 0.6499999761581421, 0.5849999785423279, 0.6349999904632568, 0.6200000047683716, 0.6299999952316284, 0.7250000238418579, 0.6949999928474426, 0.7250000238418579, 0.6949999928474426, 0.7400000095367432, 0.675000011920929, 0.7250000238418579, 0.7850000262260437, 0.7149999737739563, 0.7250000238418579, 0.7850000262260437, 0.7599999904632568, 0.7900000214576721, 0.7599999904632568, 0.7850000262260437, 0.8050000071525574, 0.7699999809265137, 0.8349999785423279, 0.7749999761581421, 0.8450000286102295, 0.8100000023841858, 0.7450000047683716, 0.7699999809265137, 0.8100000023841858, 0.8349999785423279, 0.8050000071525574, 0.8650000095367432, 0.8349999785423279, 0.8450000286102295, 0.8299999833106995, 0.8450000286102295, 0.8399999737739563, 0.8399999737739563, 0.800000011920929, 0.8550000190734863, 0.8650000095367432, 0.8600000143051147, 0.8799999952316284, 0.875, 0.8650000095367432, 0.9100000262260437, 0.8650000095367432, 0.8849999904632568, 0.8999999761581421, 0.8799999952316284, 0.8799999952316284, 0.9049999713897705, 0.8999999761581421, 0.8899999856948853, 0.9049999713897705, 0.8700000047683716, 0.8899999856948853, 0.9100000262260437, 0.8949999809265137, 0.9150000214576721, 0.8849999904632568, 0.9049999713897705, 0.8700000047683716, 0.9350000023841858, 0.9300000071525574, 0.9100000262260437, 0.9200000166893005, 0.9150000214576721, 0.9150000214576721, 0.9549999833106995, 0.8799999952316284, 0.949999988079071, 0.925000011920929, 0.9150000214576721, 0.9200000166893005, 0.9049999713897705, 0.9300000071525574, 0.9399999976158142, 0.8899999856948853, 0.925000011920929, 0.9100000262260437, 0.9100000262260437, 0.9150000214576721, 0.949999988079071, 0.9300000071525574, 0.9100000262260437, 0.9449999928474426, 0.9200000166893005, 0.949999988079071, 0.925000011920929, 0.9200000166893005, 0.9200000166893005, 0.925000011920929, 0.9350000023841858, 0.9549999833106995, 0.949999988079071, 0.9300000071525574, 0.9200000166893005, 0.9399999976158142, 0.9300000071525574, 0.9449999928474426, 0.9449999928474426, 0.9300000071525574, 0.9599999785423279, 0.9399999976158142, 0.9350000023841858, 0.925000011920929, 0.9750000238418579, 0.9599999785423279, 0.9449999928474426, 0.9549999833106995, 0.9750000238418579, 0.9549999833106995, 0.9449999928474426, 0.9549999833106995, 0.949999988079071, 0.9449999928474426, 0.9599999785423279, 0.9549999833106995, 0.9300000071525574, 0.9449999928474426, 0.9649999737739563, 0.9900000095367432, 0.949999988079071, 0.9549999833106995, 0.925000011920929, 0.9750000238418579, 0.9350000023841858, 0.9350000023841858, 0.949999988079071, 0.9549999833106995, 0.9399999976158142, 0.9599999785423279, 0.949999988079071, 0.9549999833106995, 0.9700000286102295, 0.9599999785423279, 0.949999988079071, 0.9599999785423279, 0.9649999737739563, 0.9599999785423279, 0.9800000190734863, 0.949999988079071, 0.9700000286102295, 0.949999988079071, 0.9599999785423279, 0.9800000190734863, 0.9350000023841858, 0.9700000286102295, 0.9599999785423279, 0.9649999737739563, 0.949999988079071, 0.949999988079071, 0.9750000238418579, 0.949999988079071, 0.9399999976158142, 0.9599999785423279, 0.9649999737739563, 0.9800000190734863, 0.9649999737739563, 0.9649999737739563, 0.9750000238418579, 0.9599999785423279, 0.9449999928474426, 0.9750000238418579, 0.9700000286102295, 0.9750000238418579, 0.9649999737739563, 0.9649999737739563, 0.9800000190734863, 0.9649999737739563, 0.9750000238418579, 0.9750000238418579, 0.9700000286102295, 0.9850000143051147, 0.9700000286102295, 0.9800000190734863, 0.9750000238418579, 0.9649999737739563, 0.9700000286102295, 0.9649999737739563, 0.9850000143051147, 0.9800000190734863, 0.9750000238418579, 0.9599999785423279, 0.9649999737739563, 0.9750000238418579, 0.9700000286102295, 0.9750000238418579, 0.9800000190734863, 0.9750000238418579, 0.9599999785423279, 0.9850000143051147, 0.9750000238418579, 0.9649999737739563, 0.9549999833106995, 0.9950000047683716, 0.9950000047683716, 0.9800000190734863, 0.9800000190734863, 0.9750000238418579, 0.9800000190734863, 0.9750000238418579, 0.9800000190734863, 0.9800000190734863, 0.9750000238418579, 0.9649999737739563, 0.9700000286102295, 0.9850000143051147, 0.9649999737739563, 0.9700000286102295, 0.9700000286102295, 0.9950000047683716, 0.9750000238418579, 0.9800000190734863, 0.9750000238418579, 0.9750000238418579, 0.9850000143051147, 0.9800000190734863, 0.9750000238418579, 0.9800000190734863, 0.9800000190734863, 0.9800000190734863, 0.9800000190734863, 0.9850000143051147, 0.9950000047683716, 0.9800000190734863, 0.9800000190734863, 0.9750000238418579, 0.9649999737739563, 0.9950000047683716, 0.9900000095367432, 0.9850000143051147, 0.9850000143051147, 0.9950000047683716, 0.9950000047683716, 0.9850000143051147, 0.9850000143051147, 0.9900000095367432, 0.9850000143051147, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9850000143051147, 0.9950000047683716, 0.9900000095367432, 0.9950000047683716, 0.9850000143051147, 0.9850000143051147, 1.0, 0.9850000143051147, 0.9800000190734863, 0.9800000190734863, 0.9950000047683716, 0.9950000047683716, 0.9850000143051147, 1.0, 0.9900000095367432, 0.9850000143051147, 0.9850000143051147, 0.9750000238418579, 0.9800000190734863, 0.9700000286102295, 0.9950000047683716, 0.9800000190734863, 0.9700000286102295, 0.9800000190734863, 0.9950000047683716, 0.9900000095367432, 0.9800000190734863, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9750000238418579, 1.0, 1.0, 0.9800000190734863, 0.9950000047683716, 0.9750000238418579, 0.9850000143051147, 0.9750000238418579, 0.9800000190734863, 0.9850000143051147, 0.9800000190734863, 1.0, 0.9950000047683716, 0.9800000190734863, 0.9800000190734863, 0.9850000143051147, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 1.0, 0.9900000095367432, 0.9850000143051147, 0.9850000143051147, 0.9800000190734863, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 0.9900000095367432, 1.0, 0.9950000047683716, 0.9850000143051147, 0.9950000047683716, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9850000143051147, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 0.9900000095367432, 0.9900000095367432, 0.9800000190734863, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9900000095367432, 0.9850000143051147, 0.9950000047683716, 0.9850000143051147, 0.9850000143051147, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9800000190734863, 0.9950000047683716, 0.9850000143051147, 0.9950000047683716, 0.9900000095367432, 0.9850000143051147, 1.0, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9850000143051147, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 1.0, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9900000095367432, 1.0, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9900000095367432, 0.9950000047683716, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9900000095367432, 0.9850000143051147, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9850000143051147, 0.9900000095367432, 0.9950000047683716, 0.9900000095367432, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 0.9950000047683716, 1.0, 1.0, 0.9950000047683716, 0.9950000047683716, 1.0, 0.9950000047683716, 0.9950000047683716, 0.9900000095367432, 1.0, 1.0, 0.9900000095367432, 1.0, 0.9900000095367432, 1.0, 0.9950000047683716, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9900000095367432, 0.9950000047683716, 1.0, 0.9850000143051147, 0.9900000095367432, 0.9900000095367432, 1.0, 0.9950000047683716, 0.9900000095367432, 1.0, 0.9950000047683716, 1.0], 'loss': [12.284319877624512, 11.101275444030762, 10.141792297363281, 9.47555923461914, 9.00936508178711, 8.590282440185547, 8.353398323059082, 8.055129051208496, 7.665520668029785, 7.521552085876465, 7.303418159484863, 6.911564826965332, 6.8104424476623535, 6.731997013092041, 6.677556991577148, 6.311470985412598, 6.262608528137207, 5.976608753204346, 5.928034782409668, 5.895518779754639, 5.677485942840576, 5.541707992553711, 5.476693153381348, 5.563443183898926, 5.257396221160889, 5.038669586181641, 4.993391036987305, 4.807231426239014, 4.772585868835449, 4.396142482757568, 4.39601469039917, 4.350811004638672, 4.145881652832031, 4.018322467803955, 4.143087863922119, 3.6981067657470703, 3.568467617034912, 3.9105660915374756, 3.7222707271575928, 3.403315782546997, 3.317167043685913, 3.3262805938720703, 3.0648865699768066, 3.061474084854126, 2.955761671066284, 3.214632034301758, 2.612189292907715, 2.7298173904418945, 2.5909430980682373, 2.543546676635742, 2.766159772872925, 2.3671534061431885, 2.2455103397369385, 2.3008363246917725, 2.200195789337158, 2.336233615875244, 2.166731834411621, 2.221747636795044, 2.1394453048706055, 2.004772663116455, 1.9530311822891235, 1.8759194612503052, 1.9243862628936768, 1.61948823928833, 1.7766032218933105, 1.815250039100647, 1.7424534559249878, 1.7231882810592651, 1.6025499105453491, 1.492279052734375, 1.4951739311218262, 1.4183441400527954, 1.4011033773422241, 1.4248642921447754, 1.3229658603668213, 1.2808297872543335, 1.172948956489563, 1.403525948524475, 1.2427470684051514, 1.1639347076416016, 1.2851357460021973, 1.3551127910614014, 1.1791831254959106, 1.0907024145126343, 1.1762531995773315, 1.0397955179214478, 1.0396713018417358, 1.0195493698120117, 0.9932741522789001, 1.0009405612945557, 0.9886954426765442, 1.0975241661071777, 1.0751326084136963, 0.9866474270820618, 0.8850266933441162, 0.8621817827224731, 0.8860973119735718, 0.9513969421386719, 0.8633450865745544, 0.65626060962677, 0.8961922526359558, 0.8394484519958496, 0.8338454961776733, 0.8087608814239502, 0.7558143138885498, 0.7435107231140137, 0.7413569688796997, 0.6369960904121399, 0.6146056652069092, 0.8817435503005981, 0.6680035591125488, 0.6973269581794739, 0.7367948889732361, 0.6139234900474548, 0.7177917957305908, 0.6018133163452148, 0.6755573153495789, 0.6412884593009949, 0.5288724303245544, 0.6621130108833313, 0.5614594221115112, 0.5988865494728088, 0.5531634092330933, 0.4837568700313568, 0.6285831928253174, 0.5721639394760132, 0.6017444729804993, 0.5707988739013672, 0.5168134570121765, 0.5976943969726562, 0.5198692083358765, 0.5167508125305176, 0.5716641545295715, 0.5284345746040344, 0.537747859954834, 0.5532971620559692, 0.5507543683052063, 0.4358583688735962, 0.41735589504241943, 0.532548189163208, 0.42878392338752747, 0.4604432284832001, 0.47551825642585754, 0.5398125648498535, 0.4883161783218384, 0.48908165097236633, 0.39715519547462463, 0.4397197961807251, 0.48855045437812805, 0.36191558837890625, 0.43386635184288025, 0.5318718552589417, 0.4228576421737671, 0.4418814480304718, 0.39324846863746643, 0.33833321928977966, 0.381195604801178, 0.287941575050354, 0.3690015375614166, 0.3993054926395416, 0.42119601368904114, 0.3141903281211853, 0.30260947346687317, 0.3179609179496765, 0.31485655903816223, 0.3083474636077881, 0.26646503806114197, 0.3728061616420746, 0.30612805485725403, 0.3310811519622803, 0.43493175506591797, 0.2795329689979553, 0.3346824645996094, 0.335660845041275, 0.2681652307510376, 0.25082552433013916, 0.29446941614151, 0.36573079228401184, 0.29832756519317627, 0.36827772855758667, 0.3456379771232605, 0.27608346939086914, 0.3455962836742401, 0.2950696051120758, 0.32082200050354004, 0.3359394371509552, 0.2969335615634918, 0.28758853673934937, 0.2838621437549591, 0.1961294561624527, 0.26404544711112976, 0.28914397954940796, 0.2089250236749649, 0.21245120465755463, 0.2735691964626312, 0.17932488024234772, 0.2978943884372711, 0.19876153767108917, 0.36876749992370605, 0.24212488532066345, 0.17336200177669525, 0.30782052874565125, 0.2008398175239563, 0.24593661725521088, 0.2110072672367096, 0.21990346908569336, 0.26920655369758606, 0.23085401952266693, 0.23486602306365967, 0.28863003849983215, 0.21346180140972137, 0.1942645013332367, 0.23019474744796753, 0.14486707746982574, 0.15991975367069244, 0.1575620025396347, 0.17385876178741455, 0.23661859333515167, 0.13268479704856873, 0.18799377977848053, 0.16053390502929688, 0.17428332567214966, 0.17237588763237, 0.10740522295236588, 0.15689413249492645, 0.13669829070568085, 0.15320415794849396, 0.1660146564245224, 0.18511976301670074, 0.15135714411735535, 0.19124050438404083, 0.15550842881202698, 0.22035962343215942, 0.2170923799276352, 0.18528297543525696, 0.16064637899398804, 0.1555986851453781, 0.13393689692020416, 0.19611455500125885, 0.20088902115821838, 0.13857918977737427, 0.152492955327034, 0.14095903933048248, 0.11377620697021484, 0.12784194946289062, 0.18571650981903076, 0.13480567932128906, 0.13525493443012238, 0.16532640159130096, 0.17948973178863525, 0.12707263231277466, 0.13391751050949097, 0.14967948198318481, 0.13662581145763397, 0.14133962988853455, 0.11408206820487976, 0.15834955871105194, 0.15153871476650238, 0.10979001969099045, 0.10564582049846649, 0.21410539746284485, 0.1439175009727478, 0.10862527787685394, 0.17995642125606537, 0.13836582005023956, 0.12321076542139053, 0.11686872690916061, 0.17417122423648834, 0.14845962822437286, 0.11865308880805969, 0.11254256963729858, 0.0911831185221672, 0.16087761521339417, 0.18591219186782837, 0.07740365713834763, 0.1291045993566513, 0.09580367058515549, 0.18292227387428284, 0.12350895255804062, 0.07470197230577469, 0.10833818465471268, 0.10963064432144165, 0.12235521525144577, 0.12164881825447083, 0.1571364402770996, 0.08461183309555054, 0.12834268808364868, 0.13432852923870087, 0.08009167015552521, 0.1087888553738594, 0.06623902171850204, 0.07424648106098175, 0.070649154484272, 0.11730707436800003, 0.09581764042377472, 0.06021218001842499, 0.06467631459236145, 0.08317660540342331, 0.048116836696863174, 0.07454618066549301, 0.06075902283191681, 0.12711454927921295, 0.0879177674651146, 0.04690021649003029, 0.05689061060547829, 0.09262525290250778, 0.10138416290283203, 0.035562578588724136, 0.09476897865533829, 0.17693409323692322, 0.0586237795650959, 0.07590531557798386, 0.10570259392261505, 0.07283243536949158, 0.11086442321538925, 0.08962979912757874, 0.11967284977436066, 0.06697733700275421, 0.12939582765102386, 0.0846635028719902, 0.1069362536072731, 0.03591889142990112, 0.06416504830121994, 0.06930612027645111, 0.11041519790887833, 0.05037754029035568, 0.0520385205745697, 0.0655602291226387, 0.14200752973556519, 0.049441367387771606, 0.048273876309394836, 0.04982656612992287, 0.04885866492986679, 0.08909808844327927, 0.19109462201595306, 0.14636115729808807, 0.07006457448005676, 0.08462486416101456, 0.05891728773713112, 0.061202436685562134, 0.04006111994385719, 0.0995367169380188, 0.10865064710378647, 0.05107809603214264, 0.08831513673067093, 0.04269741475582123, 0.06053786724805832, 0.03972350433468819, 0.030851153656840324, 0.06464119255542755, 0.08094660937786102, 0.08513174206018448, 0.07258068025112152, 0.05981471389532089, 0.047651056200265884, 0.029918236657977104, 0.03364760801196098, 0.06117432564496994, 0.03080609254539013, 0.07750988006591797, 0.0483873076736927, 0.03658895194530487, 0.06491833925247192, 0.06452556699514389, 0.041314128786325455, 0.08509106934070587, 0.04174324870109558, 0.057140249758958817, 0.06106667220592499, 0.04427438601851463, 0.05207870155572891, 0.01789671927690506, 0.028338909149169922, 0.045428041368722916, 0.05680257827043533, 0.06562712043523788, 0.04615284129977226, 0.025116313248872757, 0.020075611770153046, 0.03198127821087837, 0.0294013861566782, 0.036949027329683304, 0.009088417515158653, 0.042245496064424515, 0.034125279635190964, 0.017416810616850853, 0.05202838033437729, 0.06122317910194397, 0.07930620014667511, 0.04390871897339821, 0.0719706267118454, 0.08635687083005905, 0.02102717198431492, 0.0438055545091629, 0.014233133755624294, 0.00878057163208723, 0.05095937103033066, 0.06928112357854843, 0.04140857607126236, 0.0518837608397007, 0.04379010200500488, 0.06510023027658463, 0.026801498606801033, 0.033335793763399124, 0.041540227830410004, 0.040483810007572174, 0.042163729667663574, 0.019484631717205048, 0.015933146700263023, 0.03158630058169365, 0.022516904398798943, 0.026754414662718773, 0.024316590279340744, 0.0452723391354084, 0.023047303780913353, 0.015396407805383205, 0.056729670614004135, 0.030433371663093567, 0.04074389487504959, 0.011639266274869442, 0.045859068632125854, 0.027700312435626984, 0.05339594930410385, 0.017887139692902565, 0.065639927983284, 0.02049238607287407, 0.018651608377695084, 0.02103698067367077, 0.020252585411071777, 0.016031259670853615, 0.019449684768915176, 0.025357086211442947, 0.027330951765179634, 0.025286056101322174, 0.014310525730252266, 0.02672627381980419, 0.026275549083948135, 0.013863635249435902, 0.049576178193092346, 0.0156511589884758, 0.04631470516324043, 0.028129994869232178, 0.022229738533496857, 0.02295132726430893, 0.020581698045134544, 0.04914877936244011, 0.014612298458814621, 0.013497956097126007, 0.032914627343416214, 0.0400870218873024, 0.019399475306272507, 0.02345576137304306, 0.052978962659835815, 0.004019648302346468, 0.020707735791802406, 0.028020642697811127, 0.038834959268569946, 0.048712048679590225, 0.014358125627040863, 0.02418065071105957, 0.014669852331280708, 0.016660097986459732, 0.005495928693562746, 0.021926213055849075, 0.0185505710542202, 0.017630312591791153, 0.0340837761759758, 0.020564064383506775, 0.012063844129443169, 0.012830981984734535, 0.030005982145667076, 0.020928218960762024, 0.009745701216161251, 0.025062406435608864, 0.045432768762111664, 0.029080912470817566, 0.012740220874547958, 0.009553145617246628, 0.028674473986029625, 0.005649765022099018, 0.042079586535692215, 0.015860797837376595, 0.02589656412601471, 0.005920493509620428, 0.021837584674358368, 0.01232950296252966, 0.015632564201951027, 0.0029283189214766026, 0.02025916427373886, 0.026475071907043457, 0.026135502383112907, 0.08884304761886597, 0.054219283163547516, 0.04081869497895241, 0.011316163465380669, 0.02643200010061264, 0.024860195815563202, 0.013947082683444023, 0.058286432176828384, 0.013687890954315662], 'val_activation_1_accuracy': [0.0, 0.15000000596046448, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.15000000596046448, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.25, 0.20000000298023224, 0.20000000298023224, 0.25, 0.20000000298023224, 0.25, 0.25, 0.25, 0.15000000596046448, 0.20000000298023224, 0.20000000298023224, 0.15000000596046448, 0.15000000596046448, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.25, 0.25, 0.25, 0.25, 0.3499999940395355, 0.3499999940395355, 0.4000000059604645, 0.3499999940395355, 0.3499999940395355, 0.3499999940395355, 0.3499999940395355, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.44999998807907104, 0.4000000059604645, 0.4000000059604645, 0.44999998807907104, 0.44999998807907104, 0.44999998807907104, 0.44999998807907104, 0.5, 0.44999998807907104, 0.44999998807907104, 0.5, 0.5, 0.5, 0.5, 0.550000011920929, 0.5, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.5, 0.44999998807907104, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.5, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.5, 0.550000011920929, 0.5, 0.5, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6000000238418579, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.6499999761581421, 0.6000000238418579, 0.699999988079071, 0.699999988079071, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.75, 0.75, 0.75, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071], 'val_activation_accuracy': [0.05000000074505806, 0.05000000074505806, 0.10000000149011612, 0.15000000596046448, 0.10000000149011612, 0.05000000074505806, 0.05000000074505806, 0.05000000074505806, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.10000000149011612, 0.15000000596046448, 0.20000000298023224, 0.20000000298023224, 0.20000000298023224, 0.25, 0.20000000298023224, 0.25, 0.25, 0.25, 0.25, 0.30000001192092896, 0.25, 0.3499999940395355, 0.3499999940395355, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.44999998807907104, 0.44999998807907104, 0.44999998807907104, 0.44999998807907104, 0.4000000059604645, 0.4000000059604645, 0.4000000059604645, 0.44999998807907104, 0.44999998807907104, 0.44999998807907104, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.550000011920929, 0.5, 0.5, 0.5, 0.5, 0.550000011920929, 0.5, 0.5, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.5, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.550000011920929, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.550000011920929, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.75, 0.75, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.75, 0.75, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.75, 0.75, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.550000011920929, 0.550000011920929, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.75, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.550000011920929, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.550000011920929, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.699999988079071, 0.699999988079071, 0.6000000238418579, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6000000238418579, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6000000238418579, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.6499999761581421, 0.699999988079071, 0.699999988079071, 0.75, 0.75, 0.699999988079071, 0.75, 0.75, 0.75, 0.75], 'val_loss': [10.73967456817627, 9.551263809204102, 8.74678897857666, 8.149347305297852, 7.786184787750244, 7.509490013122559, 7.288031578063965, 7.100883483886719, 6.974674224853516, 6.817385196685791, 6.734858512878418, 6.6507062911987305, 6.571089267730713, 6.507816314697266, 6.430732727050781, 6.361807346343994, 6.297183036804199, 6.230663299560547, 6.118067741394043, 6.147116184234619, 6.120126724243164, 6.025092124938965, 5.969494819641113, 5.969906806945801, 5.990433692932129, 5.9766669273376465, 5.941227912902832, 5.904244422912598, 5.869913578033447, 5.836706638336182, 5.750840187072754, 5.721974849700928, 5.660824775695801, 5.6819915771484375, 5.696744918823242, 5.686434268951416, 5.644425392150879, 5.626707077026367, 5.636260032653809, 5.697400093078613, 5.651261806488037, 5.614380359649658, 5.549620628356934, 5.422390937805176, 5.481512546539307, 5.387765407562256, 5.447165489196777, 5.398667335510254, 5.389830589294434, 5.3041486740112305, 5.347707271575928, 5.43133544921875, 5.3840837478637695, 5.492499351501465, 5.481583595275879, 5.361226558685303, 5.2823686599731445, 5.371223449707031, 5.308323860168457, 5.477452278137207, 5.52101993560791, 5.562158584594727, 5.634408473968506, 5.492018699645996, 5.516043663024902, 5.34417724609375, 5.349291801452637, 5.251215934753418, 5.266229152679443, 5.286649703979492, 5.4120306968688965, 5.453193187713623, 5.5009918212890625, 5.405852317810059, 5.558281898498535, 5.567016124725342, 5.5865960121154785, 5.540165901184082, 5.522412300109863, 5.657384395599365, 5.354918479919434, 5.258581161499023, 5.510851860046387, 5.593067169189453, 5.327681064605713, 5.289881229400635, 5.367114067077637, 5.295029163360596, 5.4378814697265625, 5.191928863525391, 5.326813697814941, 4.961708068847656, 5.068917274475098, 5.066777229309082, 5.136915683746338, 5.057458877563477, 5.034384727478027, 4.902225494384766, 4.739195823669434, 5.0310797691345215, 4.737457752227783, 4.735785007476807, 4.737046718597412, 4.7969865798950195, 4.5504279136657715, 4.683825492858887, 4.672979831695557, 4.795373439788818, 4.7431960105896, 4.6095147132873535, 4.827627658843994, 4.781312465667725, 4.640659332275391, 4.968937873840332, 4.875539779663086, 5.015175819396973, 4.869475364685059, 4.8882317543029785, 4.730987548828125, 4.682188510894775, 4.988158226013184, 4.956450462341309, 4.987781524658203, 4.9541215896606445, 5.037220001220703, 5.0838823318481445, 5.081794738769531, 4.899275302886963, 4.954695701599121, 4.932889938354492, 4.581368446350098, 4.856045246124268, 4.563307762145996, 4.6095709800720215, 4.594857215881348, 4.717584133148193, 4.6819586753845215, 4.688253879547119, 4.600543022155762, 4.657495021820068, 4.778785705566406, 4.802833557128906, 4.569070339202881, 4.5694684982299805, 4.678163051605225, 4.6577067375183105, 4.593347072601318, 4.477872371673584, 4.2610015869140625, 4.50195837020874, 4.616034507751465, 4.669798851013184, 4.688388824462891, 4.297342777252197, 4.430856227874756, 4.647107124328613, 4.52158260345459, 4.631760597229004, 4.572641849517822, 4.725348472595215, 4.344561576843262, 4.561741352081299, 4.754898548126221, 4.774853706359863, 4.97982120513916, 4.823031902313232, 4.729890823364258, 4.42391300201416, 4.362515449523926, 4.522988796234131, 4.616443157196045, 4.842036247253418, 4.771790981292725, 4.472940444946289, 4.952255725860596, 4.768080711364746, 5.022622108459473, 4.5891594886779785, 4.500267505645752, 4.709368705749512, 4.3761725425720215, 4.583934783935547, 4.458233833312988, 4.632565498352051, 4.627851486206055, 4.53219747543335, 4.583880424499512, 4.62514591217041, 4.680427551269531, 4.8136796951293945, 4.682860374450684, 4.456104278564453, 4.743997097015381, 4.772854804992676, 4.660330772399902, 4.761044502258301, 4.909064292907715, 4.9203691482543945, 4.457653999328613, 4.573746681213379, 4.703859806060791, 4.6996073722839355, 4.9127349853515625, 4.826704502105713, 5.036640644073486, 4.738881587982178, 4.681487560272217, 4.654514312744141, 4.502453327178955, 4.769331932067871, 4.940794467926025, 4.70996618270874, 4.739529609680176, 4.978498935699463, 5.287203788757324, 5.246962547302246, 5.060341835021973, 4.847450256347656, 4.983754634857178, 4.987494945526123, 5.024964809417725, 4.949428081512451, 5.0479021072387695, 5.221246719360352, 5.183830738067627, 5.212161064147949, 5.176381587982178, 5.040410995483398, 4.885797500610352, 5.175570011138916, 4.8600664138793945, 4.888773441314697, 5.073430061340332, 4.916263580322266, 5.212290287017822, 4.788218975067139, 4.986586570739746, 5.386139869689941, 5.405973434448242, 5.180185317993164, 5.279476165771484, 4.595631122589111, 4.868902683258057, 4.99137020111084, 4.7970290184021, 4.71105432510376, 4.830779075622559, 4.994692802429199, 4.855600357055664, 4.414792060852051, 4.722045421600342, 4.6897501945495605, 4.322136878967285, 4.451817035675049, 4.855517864227295, 4.683535575866699, 4.330390930175781, 4.4392266273498535, 4.549160957336426, 4.555270195007324, 4.319460868835449, 4.3243408203125, 4.626687049865723, 4.604020595550537, 4.971630096435547, 4.793910980224609, 4.684504508972168, 4.509467124938965, 4.627378940582275, 4.83129358291626, 4.9903669357299805, 4.921548843383789, 4.39578104019165, 4.5090203285217285, 4.809296607971191, 5.1131181716918945, 5.36477518081665, 4.816595554351807, 4.357444763183594, 4.46333122253418, 4.5457234382629395, 4.409838676452637, 4.297282695770264, 4.317843914031982, 4.685996055603027, 4.759631156921387, 5.1898393630981445, 4.8826165199279785, 4.5778303146362305, 4.358554363250732, 4.513760566711426, 4.617453098297119, 4.703677177429199, 4.743745803833008, 4.763577461242676, 4.918475151062012, 5.022650241851807, 4.816684722900391, 4.823288917541504, 4.729870319366455, 4.848945617675781, 4.791693210601807, 4.764139652252197, 4.633197784423828, 4.788451194763184, 4.831178188323975, 4.603798866271973, 4.595328330993652, 4.439515113830566, 4.26928186416626, 4.365178108215332, 4.6245222091674805, 4.6536865234375, 4.813791275024414, 4.514493942260742, 3.858203411102295, 4.126776218414307, 4.372488975524902, 3.9550395011901855, 4.026603698730469, 4.273718357086182, 4.25905704498291, 4.320202350616455, 4.650582313537598, 4.58900260925293, 4.647313594818115, 4.57991886138916, 4.464291095733643, 4.27266263961792, 4.5683794021606445, 4.925000190734863, 4.897214412689209, 4.562226295471191, 4.558587551116943, 4.168686866760254, 4.415824890136719, 4.641680717468262, 4.3692946434021, 4.41795015335083, 4.5788774490356445, 4.500601768493652, 4.579936981201172, 4.660614013671875, 4.529813289642334, 4.717189311981201, 4.743854522705078, 4.611392974853516, 4.526574611663818, 4.813248634338379, 4.8510355949401855, 4.254737854003906, 4.532713890075684, 4.623693466186523, 4.798895359039307, 4.907600402832031, 4.924417972564697, 5.161588668823242, 4.826825141906738, 5.01669454574585, 4.827044486999512, 4.978130340576172, 5.05324125289917, 4.9856743812561035, 4.8812360763549805, 4.742857456207275, 5.024624824523926, 5.0712480545043945, 5.000301361083984, 4.76365327835083, 4.794791221618652, 4.547796249389648, 4.508997440338135, 4.698088645935059, 4.903249263763428, 4.905003070831299, 5.065325736999512, 5.044790744781494, 4.686491966247559, 4.540600776672363, 4.616535186767578, 4.774020195007324, 4.906843662261963, 4.998275279998779, 4.974687576293945, 5.047277450561523, 5.070059299468994, 4.768564701080322, 4.782797336578369, 5.009276866912842, 4.855201721191406, 4.69451379776001, 5.068123817443848, 5.02808952331543, 5.06013298034668, 5.200199127197266, 5.2579216957092285, 4.926949501037598, 4.840252876281738, 4.837711811065674, 5.266742706298828, 5.039536476135254, 4.765750885009766, 4.511542320251465, 4.730920314788818, 5.009699821472168, 5.083193778991699, 4.737063884735107, 4.6934099197387695, 4.775070667266846, 4.999869346618652, 5.183833122253418, 5.046238899230957, 4.896489143371582, 4.8048248291015625, 4.778550148010254, 4.680209636688232, 4.721856594085693, 4.609194755554199, 4.695274353027344, 4.797297477722168, 4.544649600982666, 4.740913391113281, 4.827670097351074, 4.654102802276611, 4.937407493591309, 4.984963417053223, 4.89260196685791, 4.9688262939453125, 4.982726573944092, 4.986281394958496, 4.986124515533447, 4.833449840545654, 4.540022850036621, 4.576222896575928, 4.702108383178711, 4.604275703430176, 4.724632263183594, 4.86578893661499, 4.8676581382751465, 4.903891563415527, 4.923033237457275, 4.818774223327637, 4.813042640686035, 5.022233009338379, 5.071258068084717, 5.059263706207275, 5.1173601150512695, 5.169676303863525, 5.155997276306152, 5.032634735107422, 5.148468971252441, 5.2613420486450195, 5.1973981857299805, 5.195631980895996, 5.459074974060059, 5.355910301208496, 5.540904998779297, 5.521090507507324, 5.478342533111572, 5.6435546875, 5.7093939781188965, 5.516617774963379, 5.376694679260254, 5.587121963500977, 5.665918827056885, 5.657740116119385, 5.4064459800720215, 5.621150970458984, 5.561034202575684, 5.498996734619141, 5.3684773445129395, 5.892086505889893, 6.121647834777832, 5.728480339050293, 5.780246734619141, 6.115196228027344, 6.172499179840088, 6.053179740905762, 5.869232654571533, 5.560202598571777, 5.51499605178833, 5.676902770996094, 5.478447914123535, 5.493436336517334, 5.165673732757568, 5.217899799346924, 5.393613338470459, 5.470227241516113, 5.7205095291137695, 5.589574337005615, 5.616249084472656, 5.448481559753418, 5.458390235900879, 4.914495944976807, 4.911149024963379, 5.007935523986816, 5.000013828277588, 4.862933158874512, 4.992028713226318, 5.054959297180176]}

import matplotlib.pyplot as plt
a=28
acc = history['activation_1_accuracy']#[0:a]
val_acc = history['val_activation_1_accuracy']#[0:a]
loss = history['loss']#[0:a]
val_loss = history['val_loss']#[0:a]
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
#用训练集、验证集、测试集的样本，预测
predict_result(x_train, y_train, show=False) 
predict_result(x_eval, y_eval, show=False) 
predict_result(x_test, y_test, show=True) 